In [1]:
from datetime import datetime
import pandas
from typing import List, Any
import pyspark.sql.functions as F

import query_lib
import indicator_lib

In [2]:
BASE_DIR='./test_files/parquet_big_db'
#CODE_SYSTEM='http://snomed.info/sct'
CODE_SYSTEM='http://www.ampathkenya.org'
# Note since this issue is resolved we don't need BASE_URL:
# https://github.com/GoogleCloudPlatform/openmrs-fhir-analytics/issues/55
#BASE_URL='http://localhost:8099/openmrs/ws/fhir2/R4/'
BASE_URL=''

# Encounter view

In [3]:
patient_query = query_lib.patient_query_factory(
    query_lib.Runner.SPARK, BASE_DIR, CODE_SYSTEM)

flat_enc_df = patient_query.get_patient_encounter_view(BASE_URL)
flat_enc_df.head()

[INDICATORS_LOG 2021-10-29 23:18:59.719376] Number of Patient resources= 3958
[INDICATORS_LOG 2021-10-29 23:19:02.118676] Number of Encounter resources= 199182


,encPatientId,locationId,locationDisplay,encTypeSystem,encTypeCode,num_encounters,firstDate,lastDate
0,0a96ccc4-2336-4300-bdcc-461674233514,None,None,[http://fhir.openmrs.org/code-system/encounter...,[5021b1a1-e7f6-44b4-ba02-da2f2bcf8718],36,1996-12-02T08:32:42+00:00,None
1,0b29d253-0075-4135-88b3-a2c7e1f1017c,None,None,[http://fhir.openmrs.org/code-system/encounter...,[5021b1a1-e7f6-44b4-ba02-da2f2bcf8718],79,1942-07-24T17:17:32+00:00,None
2,1cf07d21-630f-4c56-afed-6b4048176ec5,None,None,[http://fhir.openmrs.org/code-system/encounter...,[5021b1a1-e7f6-44b4-ba02-da2f2bcf8718],31,1996-04-14T19:10:21+00:00,None
3,1d60d610-cf11-42db-8a2d-9fd3a8ba80c2,None,None,[http://fhir.openmrs.org/code-system/encounter...,[5021b1a1-e7f6-44b4-ba02-da2f2bcf8718],32,1998-08-05T01:42:57+00:00,None
4,3e359d1d-c350-4369-a466-1190c726d43b,None,None,[http://fhir.openmrs.org/code-system/encounter...,[5021b1a1-e7f6-44b4-ba02-da2f2bcf8718],50,1964-04-17T14:22:01+00:00,None


In [4]:
flat_enc_df[flat_enc_df['locationId'].notna()].head()

,encPatientId,locationId,locationDisplay,encTypeSystem,encTypeCode,num_encounters,firstDate,lastDate
214,8295eb5b-fba6-4e83-a5cb-2817b135cd27,58c57d25-8d39-41ab-8422-108a0c277d98,Outpatient Clinic,[http://fhir.openmrs.org/code-system/encounter...,[67a71486-1a54-468f-ac3e-7091a9a79584],10,2019-02-04T09:59:58+00:00,None
246,637d5571-dee2-4e5e-8ece-b65d2d965081,8d6c993e-c2cc-11de-8d13-0010c6dffd0f,Unknown Location,[http://fhir.openmrs.org/code-system/encounter...,[d7151f82-c1f3-4152-a605-2f9ea7414a79],2,2020-09-27T11:13:01+00:00,None
789,637d5571-dee2-4e5e-8ece-b65d2d965081,58c57d25-8d39-41ab-8422-108a0c277d98,Outpatient Clinic,[http://fhir.openmrs.org/code-system/encounter...,[67a71486-1a54-468f-ac3e-7091a9a79584],2,2020-09-27T10:05:01+00:00,None
842,8295eb5b-fba6-4e83-a5cb-2817b135cd27,b1a8b05e-3542-4037-bbd3-998ee9c40574,Inpatient Ward,[http://fhir.openmrs.org/code-system/encounter...,[e22e39fd-7db2-45e7-80f1-60fa0d5a4378],2,2019-02-28T10:43:00+00:00,None
1640,8295eb5b-fba6-4e83-a5cb-2817b135cd27,8d6c993e-c2cc-11de-8d13-0010c6dffd0f,Unknown Location,[http://fhir.openmrs.org/code-system/encounter...,[d7151f82-c1f3-4152-a605-2f9ea7414a79],10,2019-02-04T11:09:58+00:00,None


## Adding an encounter location constraint

In [5]:
# Add encounter location constraint
patient_query.encounter_constraints(locationId=['58c57d25-8d39-41ab-8422-108a0c277d98'])
flat_enc_df = patient_query.get_patient_encounter_view(BASE_URL)
flat_enc_df.head()

,encPatientId,locationId,locationDisplay,encTypeSystem,encTypeCode,num_encounters,firstDate,lastDate
0,8295eb5b-fba6-4e83-a5cb-2817b135cd27,58c57d25-8d39-41ab-8422-108a0c277d98,Outpatient Clinic,[http://fhir.openmrs.org/code-system/encounter...,[67a71486-1a54-468f-ac3e-7091a9a79584],10,2019-02-04T09:59:58+00:00,None
1,637d5571-dee2-4e5e-8ece-b65d2d965081,58c57d25-8d39-41ab-8422-108a0c277d98,Outpatient Clinic,[http://fhir.openmrs.org/code-system/encounter...,[67a71486-1a54-468f-ac3e-7091a9a79584],2,2020-09-27T10:05:01+00:00,None


In [6]:
flat_enc_df[flat_enc_df['encPatientId'] == '8295eb5b-fba6-4e83-a5cb-2817b135cd27']

,encPatientId,locationId,locationDisplay,encTypeSystem,encTypeCode,num_encounters,firstDate,lastDate
0,8295eb5b-fba6-4e83-a5cb-2817b135cd27,58c57d25-8d39-41ab-8422-108a0c277d98,Outpatient Clinic,[http://fhir.openmrs.org/code-system/encounter...,[67a71486-1a54-468f-ac3e-7091a9a79584],10,2019-02-04T09:59:58+00:00,None


In [7]:
flat_enc = patient_query._flatten_encounter('')
flat_enc.head().asDict()

{'encounterId': 'c940b82f-5179-4f7f-8613-6b3b800d9ac1',
 'encPatientId': '8295eb5b-fba6-4e83-a5cb-2817b135cd27',
 'first': '2019-04-13T09:39:56+00:00',
 'last': None,
 'locationId': '58c57d25-8d39-41ab-8422-108a0c277d98',
 'locationDisplay': 'Outpatient Clinic',
 'encTypeSystem': ['http://fhir.openmrs.org/code-system/encounter-type'],
 'encTypeCode': ['67a71486-1a54-468f-ac3e-7091a9a79584']}

# Observation view

In [8]:
_VL_CODE = '856'  # HIV VIRAL LOAD
_ARV_PLAN = '1255'  # ANTIRETROVIRAL PLAN
end_date='2018-01-01'
start_date='1998-01-01'
old_start_date='1978-01-01'

# Creating a new `patient_query` to drop all previous constraints
# and recreate flat views.
patient_query = query_lib.patient_query_factory(
    query_lib.Runner.SPARK, BASE_DIR, CODE_SYSTEM)

patient_query.include_obs_values_in_time_range(
    _VL_CODE, min_time=start_date, max_time=end_date)
patient_query.include_obs_values_in_time_range(
    _ARV_PLAN, min_time=start_date, max_time=end_date)
patient_query.include_all_other_codes(min_time=start_date, max_time=end_date)
# Note the first call to `find_patient_aggregates` starts a local Spark
# cluster, load input files, and flattens observations. These won't be
# done in subsequent calls of this function on the same instance.
# Also same cluster will be reused for other instances of `PatientQuery`.
agg_df = patient_query.get_patient_obs_view(BASE_URL)
agg_df.head(10)

[INDICATORS_LOG 2021-10-29 23:19:16.793786] Number of Patient resources= 3958
[INDICATORS_LOG 2021-10-29 23:19:17.586591] Number of Observation resources= 860105
[INDICATORS_LOG 2021-10-29 23:19:21.464072] Number of flattened obs rows = 1275285
[INDICATORS_LOG 2021-10-29 23:19:21.862229] Number of Encounter resources= 199182
[INDICATORS_LOG 2021-10-29 23:19:29.209789] Number of aggregated obs= 51312
[INDICATORS_LOG 2021-10-29 23:19:34.074160] Number of joined patient_agg_obs= 51312
[INDICATORS_LOG 2021-10-29 23:19:46.213116] patient_obs_view size= 51312


,patientId,birthDate,gender,code,num_obs,min_value,max_value,min_date,max_date,first_value,last_value,first_value_code,last_value_code
0,009b3fce-f62e-4308-bace-594afa08aeee,1989-04-09,male,5085,4,108.0,243.0,2005-08-08T22:42:08+00:00,2006-11-11T22:42:08+00:00,108.0,243.0,None,None
1,010f6478-cf83-4372-9adb-3bdc563fc4f6,1994-02-16,male,856,1,471400.0,471400.0,2010-04-08T11:19:37+00:00,2010-04-08T11:19:37+00:00,471400.0,471400.0,None,None
2,0389d24e-d0e6-4b27-9eab-5276fd21d075,1958-09-11,male,1250,37,NaN,NaN,2007-05-17T16:32:51+00:00,2010-07-18T16:32:51+00:00,None,None,628,815
3,03a70644-ad56-4dce-9228-ad932a6b00ef,1981-07-27,male,1040,11,NaN,NaN,1999-09-28T13:20:13+00:00,2006-02-22T13:20:13+00:00,None,None,703,1304
4,0444c2c5-7e1c-42e9-b116-299cf0708d7a,1995-07-16,female,1040,9,NaN,NaN,2012-04-09T20:05:06+00:00,2017-08-19T20:05:06+00:00,None,None,1304,704
5,04b4e361-78fe-41e1-8c8a-7a77b86e8318,1944-01-27,male,1261,1,NaN,NaN,2007-08-16T19:38:04+00:00,2007-08-16T19:38:04+00:00,None,None,1260,1260
6,04fa820b-7667-4765-87ac-278be13be93e,1996-09-14,male,5088,1,37.0,37.0,2016-12-21T10:00:31+00:00,2016-12-21T10:00:31+00:00,37.0,37.0,None,None
7,0617fd95-e03e-4c3d-ba90-e602d22b6413,1960-04-02,female,1040,2,NaN,NaN,1998-03-28T08:45:05+00:00,1998-03-28T08:45:05+00:00,None,None,703,704
8,075b5adf-7dff-4252-a130-5109afd93d35,1997-06-20,female,1270,57,NaN,NaN,2013-07-23T04:19:28+00:00,2015-03-04T04:19:28+00:00,None,None,2226,768
9,080f9774-fa64-4feb-bcb0-6dee338272ef,1959-11-15,female,1111,14,NaN,NaN,2011-03-29T17:41:13+00:00,2011-03-29T17:41:13+00:00,None,None,1107,768


In [9]:
# Inspecting one specific patient.
agg_df[agg_df['patientId'] == '009b3fce-f62e-4308-bace-594afa08aeee'].head()

,patientId,birthDate,gender,code,num_obs,min_value,max_value,min_date,max_date,first_value,last_value,first_value_code,last_value_code
0,009b3fce-f62e-4308-bace-594afa08aeee,1989-04-09,male,5085,4,108.0,243.0,2005-08-08T22:42:08+00:00,2006-11-11T22:42:08+00:00,108.0,243.0,None,None
1990,009b3fce-f62e-4308-bace-594afa08aeee,1989-04-09,male,1088,16,NaN,NaN,2005-08-08T22:42:08+00:00,2006-11-11T22:42:08+00:00,None,None,6180,806
4776,009b3fce-f62e-4308-bace-594afa08aeee,1989-04-09,male,1040,3,NaN,NaN,2005-08-08T22:42:08+00:00,2006-11-11T22:42:08+00:00,None,None,703,1304
12466,009b3fce-f62e-4308-bace-594afa08aeee,1989-04-09,male,6174,9,NaN,NaN,2005-08-08T22:42:08+00:00,2006-11-11T22:42:08+00:00,None,None,157,6171
12978,009b3fce-f62e-4308-bace-594afa08aeee,1989-04-09,male,1111,28,NaN,NaN,2005-08-08T22:42:08+00:00,2006-11-11T22:42:08+00:00,None,None,1107,768


In [10]:
agg_df[(agg_df['code'] == '856') & (agg_df['min_date'] != agg_df['max_date'])][
    ['patientId', 'code', 'min_date', 'max_date', 'first_value_code', 'last_value_code']].head()

,patientId,code,min_date,max_date,first_value_code,last_value_code
13,0b583bf4-44d3-4950-9446-9064b32b98b7,856,1999-12-10T00:58:45+00:00,2007-07-16T00:58:45+00:00,None,None
17,0d807445-1f27-49a2-a12c-72fab8196d56,856,2006-11-13T19:40:56+00:00,2013-09-14T19:40:56+00:00,None,None
84,5d148707-ef04-40de-92b8-0d3a43d8b4b5,856,2014-07-07T11:20:21+00:00,2016-04-22T11:20:21+00:00,None,None
103,708d9c5b-ad60-486c-bf09-fbfea39af1ad,856,2011-06-28T09:44:45+00:00,2012-11-13T09:44:45+00:00,None,None
141,9bf28e89-4fc3-441f-9976-e4cfb01522d4,856,2000-08-08T13:47:57+00:00,2004-02-26T13:47:57+00:00,None,None


# Inspecting underlying Spark data-frames
The _user_ of the library does not need to deal with the underlying distributed query processing system. However, the _developer_ of the library needs an easy way to inspect the internal data of these systems. Here is how:

In [11]:
_DRUG1 = '1256'  # START DRUGS
_DRUG2 = '1260'  # STOP ALL MEDICATIONS

In [12]:
patient_query._obs_df.head().asDict()

{'id': '82ddd14f-734a-4f00-814c-ffc036f09e56',
 'meta': Row(id=None, versionId=None, lastUpdated=None, profile=None, security=None, tag=[Row(id=None, system='http://terminology.hl7.org/CodeSystem/v3-ObservationValue', version=None, code='SUBSETTED', display='Resource encoded in summary mode', userSelected=None)]),
 'implicitRules': None,
 'language': None,
 'text': None,
 'contained': [Row(id='#f6dc13d8-c20b-4b5b-9b32-d364e7867479', meta=Row(id=None, versionId=None, lastUpdated=None, profile=None, security=None, tag=[Row(id=None, system='http://terminology.hl7.org/CodeSystem/v3-ObservationValue', version=None, code='SUBSETTED', display='Resource encoded in summary mode', userSelected=None)]), implicitRules=None, language=None)],
 'identifier': None,
 'basedOn': None,
 'status': 'final',
 'category': None,
 'code': Row(id=None, coding=[Row(id=None, system=None, version=None, code='159911AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', display='current antiretroviral treatment, patient reported', userSe

In [13]:
exp_obs = patient_query._obs_df.withColumn('coding', F.explode('code.coding'))
exp_obs.head().asDict()

{'id': '82ddd14f-734a-4f00-814c-ffc036f09e56',
 'meta': Row(id=None, versionId=None, lastUpdated=None, profile=None, security=None, tag=[Row(id=None, system='http://terminology.hl7.org/CodeSystem/v3-ObservationValue', version=None, code='SUBSETTED', display='Resource encoded in summary mode', userSelected=None)]),
 'implicitRules': None,
 'language': None,
 'text': None,
 'contained': [Row(id='#f6dc13d8-c20b-4b5b-9b32-d364e7867479', meta=Row(id=None, versionId=None, lastUpdated=None, profile=None, security=None, tag=[Row(id=None, system='http://terminology.hl7.org/CodeSystem/v3-ObservationValue', version=None, code='SUBSETTED', display='Resource encoded in summary mode', userSelected=None)]), implicitRules=None, language=None)],
 'identifier': None,
 'basedOn': None,
 'status': 'final',
 'category': None,
 'code': Row(id=None, coding=[Row(id=None, system=None, version=None, code='159911AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', display='current antiretroviral treatment, patient reported', userSe

In [23]:
exp_obs.where('coding.code = "159800AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"').head().asDict()

{'id': 'c20527b0-98a7-470e-ab08-d6092bc2aad9',
 'meta': Row(id=None, versionId=None, lastUpdated=None, profile=None, security=None, tag=[Row(id=None, system='http://terminology.hl7.org/CodeSystem/v3-ObservationValue', version=None, code='SUBSETTED', display='Resource encoded in summary mode', userSelected=None)]),
 'implicitRules': None,
 'language': None,
 'text': None,
 'contained': [Row(id='#665a6bb1-3894-4573-9a9f-a140e3199030', meta=Row(id=None, versionId=None, lastUpdated=None, profile=None, security=None, tag=[Row(id=None, system='http://terminology.hl7.org/CodeSystem/v3-ObservationValue', version=None, code='SUBSETTED', display='Resource encoded in summary mode', userSelected=None)]), implicitRules=None, language=None)],
 'identifier': None,
 'basedOn': None,
 'status': 'final',
 'category': None,
 'code': Row(id=None, coding=[Row(id=None, system=None, version=None, code='159800AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', display='Tuberculosis, review of screening questions', userSelected=

In [25]:
exp_obs.where('coding.code = "1268"').head().asDict()

{'id': '0da7397e-1220-4d75-a9fe-645f8a67007b',
 'meta': Row(id=None, versionId=None, lastUpdated=None, profile=None, security=None, tag=[Row(id=None, system='http://terminology.hl7.org/CodeSystem/v3-ObservationValue', version=None, code='SUBSETTED', display='Resource encoded in summary mode', userSelected=None)]),
 'implicitRules': None,
 'language': None,
 'text': None,
 'contained': [Row(id='#6a149533-4743-449a-94ab-2b9319dfd15d', meta=Row(id=None, versionId=None, lastUpdated=None, profile=None, security=None, tag=[Row(id=None, system='http://terminology.hl7.org/CodeSystem/v3-ObservationValue', version=None, code='SUBSETTED', display='Resource encoded in summary mode', userSelected=None)]), implicitRules=None, language=None)],
 'identifier': None,
 'basedOn': None,
 'status': 'final',
 'category': None,
 'code': Row(id=None, coding=[Row(id=None, system=None, version=None, code='1268AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', display='Tuberculosis treatment plan', userSelected=None), Row(id=No

In [14]:
exp_obs.where(
    'coding.system IN ("http://snomed.info/sct", "http://loinc.org", "http://www.ampathkenya.org") \
    AND coding.display LIKE "%viral%" '
).groupBy(['coding']).agg(F.count('*')).head(20)

[Row(coding=Row(id=None, system='http://loinc.org', version=None, code='25836-8', display='HIV viral load', userSelected=None), count(1)=12063),
 Row(coding=Row(id=None, system='http://www.ampathkenya.org', version=None, code='1250', display='Antiretrovirals started', userSelected=None), count(1)=95137),
 Row(coding=Row(id=None, system='http://www.ampathkenya.org', version=None, code='2154', display='current antiretroviral treatment, patient reported', userSelected=None), count(1)=22639),
 Row(coding=Row(id=None, system='http://snomed.info/sct', version=None, code='315124004', display='HIV viral load', userSelected=None), count(1)=12063),
 Row(coding=Row(id=None, system='http://www.ampathkenya.org', version=None, code='1088', display='Current antiretroviral drugs used for treatment', userSelected=None), count(1)=126054),
 Row(coding=Row(id=None, system='http://snomed.info/sct', version=None, code='427314002', display='Current antiretroviral drugs used for treatment', userSelected=None)

In [15]:
agg_df[(agg_df['code'] == _ARV_PLAN) & agg_df['last_value_code'].isin([_DRUG1, _DRUG2])].head()

,patientId,birthDate,gender,code,num_obs,min_value,max_value,min_date,max_date,first_value,last_value,first_value_code,last_value_code
64,424559fc-450a-42e9-b6a1-9ebaef47cb32,1954-06-26,female,1255,1,NaN,NaN,2017-12-18T23:36:48+00:00,2017-12-18T23:36:48+00:00,None,None,1260,1260
130,8b175734-8bea-4d2f-8e92-f9829952d624,1983-10-24,female,1255,5,NaN,NaN,2004-11-29T05:26:06+00:00,2007-01-08T05:26:06+00:00,None,None,1256,1260
302,4d9da3af-e36f-4698-a681-12b385c45ea0,1997-11-15,male,1255,9,NaN,NaN,2015-03-07T02:50:33+00:00,2017-08-06T02:50:33+00:00,None,None,1107,1256
346,708d9c5b-ad60-486c-bf09-fbfea39af1ad,1981-11-20,male,1255,3,NaN,NaN,2012-11-13T09:44:45+00:00,2013-02-11T09:44:45+00:00,None,None,1107,1260
465,e6f064e8-04ac-45ed-9f6c-b8d282ea0ce1,1993-07-24,male,1255,13,NaN,NaN,2011-03-13T17:10:56+00:00,2016-07-30T17:10:56+00:00,None,None,1258,1260


In [16]:
agg_df[(agg_df['code'] == _ARV_PLAN) & agg_df['last_value_code'].isin([_DRUG1, _DRUG2])].index.size

511

In [17]:
agg_df[(agg_df['code'] == _ARV_PLAN) & agg_df['last_value_code'].isin([_DRUG1, _DRUG2])].groupby(
    'patientId').count().index.size

511

In [18]:
indicator_lib.calc_TX_NEW(agg_df, ARV_plan=_ARV_PLAN, start_drug=[_DRUG1], end_date_str=end_date)

,TX_NEW,buckets,TX_NEW_count,TX_NEW_ratio
0,False,15-19_ALL-GENDERS,136,0.063256
1,False,15-19_female,60,0.027907
2,False,15-19_male,76,0.035349
3,False,20-24_ALL-GENDERS,204,0.094884
4,False,20-24_female,105,0.048837
5,False,20-24_male,99,0.046047
6,False,25-49_ALL-GENDERS,946,0.440000
7,False,25-49_female,484,0.225116
8,False,25-49_male,462,0.214884
9,False,50+_ALL-GENDERS,512,0.238140


In [19]:
indicator_lib.calc_TX_PVLS(
    agg_df, VL_code=_VL_CODE, failure_threshold=10000,
    end_date_str=end_date)

,sup_VL,buckets,sup_VL_count,sup_VL_ratio
0,False,15-19_ALL-GENDERS,218,0.082607
1,False,15-19_female,103,0.039030
2,False,15-19_male,115,0.043577
3,False,20-24_ALL-GENDERS,335,0.126942
4,False,20-24_female,172,0.065176
5,False,20-24_male,163,0.061766
6,False,25-49_ALL-GENDERS,1378,0.522167
7,False,25-49_female,705,0.267147
8,False,25-49_male,673,0.255021
9,False,50+_ALL-GENDERS,676,0.256158


# Indicator library development
This is an example to show how the `indicator_lib.py` functions can be incrementally developed based on the query library DataFrames.

In [20]:
patient_query._flat_obs.head().asDict()

{'coding': Row(id=None, system='http://www.ampathkenya.org', version=None, code='1268', display='Tuberculosis treatment plan', userSelected=None),
 'valueCoding': Row(id=None, system='http://www.ampathkenya.org', version=None, code='1260', display='Stop all', userSelected=None),
 'value': Row(quantity=None, codeableConcept=Row(id=None, coding=[Row(id=None, system=None, version=None, code='1260AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', display='Stop all', userSelected=None), Row(id=None, system='http://www.ampathkenya.org', version=None, code='1260', display='Stop all', userSelected=None), Row(id=None, system='http://snomed.info/sct', version=None, code='274512008', display='Stop all', userSelected=None)], text=None), string=None, boolean=None, range=None, ratio=None, sampledData=None, attachment=None, time=None, dateTime=None, period=None),
 'patientId': '4054ed82-5017-430a-bb25-7603da6a70c2',
 'dateTime': '1976-07-31T04:03:01+00:00',
 'dateAndValue': '1976-07-31T04:03:01+00:00_SeP_None',
 'da

In [21]:
agg_df[(agg_df['code'] == _VL_CODE)].head()

,patientId,birthDate,gender,code,num_obs,min_value,max_value,min_date,max_date,first_value,last_value,first_value_code,last_value_code
1,010f6478-cf83-4372-9adb-3bdc563fc4f6,1994-02-16,male,856,1,471400.0,471400.0,2010-04-08T11:19:37+00:00,2010-04-08T11:19:37+00:00,471400.0,471400.0,None,None
13,0b583bf4-44d3-4950-9446-9064b32b98b7,1953-07-30,male,856,6,9305.0,981620.0,1999-12-10T00:58:45+00:00,2007-07-16T00:58:45+00:00,764440.0,981620.0,None,None
17,0d807445-1f27-49a2-a12c-72fab8196d56,1987-11-19,female,856,5,64630.0,695240.0,2006-11-13T19:40:56+00:00,2013-09-14T19:40:56+00:00,414580.0,695240.0,None,None
71,4aba5bda-4c69-4f12-8028-fda97ea50dd6,1978-02-24,male,856,1,987690.0,987690.0,2002-11-20T06:29:08+00:00,2002-11-20T06:29:08+00:00,987690.0,987690.0,None,None
80,5795144a-26f3-486f-b106-e6bb5f95b076,1963-08-16,female,856,1,170250.0,170250.0,1998-10-26T20:54:01+00:00,1998-10-26T20:54:01+00:00,170250.0,170250.0,None,None


In [22]:
def _find_age_band(birth_date: str, end_date: datetime) -> str:
  """Given the birth date, finds the age_band for PEPFAR disaggregation."""
  age = None
  try:
    # TODO handle all different formats (issues #174)
    birth = datetime.strptime(birth_date, '%Y-%m-%d')
    age = int((end_date - birth).days / 365.25)
  except Exception as e:
    common.custom_log('Invalid birth_date format: {}'.format(e))
    age = 999999

  if age == 999999:
    return 'ERROR'
  if age < 1:
    return '0-1'
  if age <= 4:
    return '1-4'
  if age <= 9:
    return '5-9'
  if age <= 14:
    return '10-14'
  if age <= 19:
    return '15-19'
  if age <= 24:
    return '20-24'
  if age <= 49:
    return '25-49'
  return '50+'


def _agg_buckets(birth_date: str, gender: str, end_date: datetime) -> List[str]:
  """Generates the list of all PEPFAR disaggregation buckets."""
  age_band = _find_age_band(birth_date, end_date)
  return [age_band + '_' + gender, 'ALL-AGES_' + gender,
          age_band + '_ALL-GENDERS', 'ALL-AGES_ALL-GENDERS']

def calc_TX_PVLS(patient_agg_obs: pandas.DataFrame, VL_code: str,
    failure_threshold: int, end_date_str: str = None) -> pandas.DataFrame:
  """Calculates TX_PVLS indicator with its corresponding disaggregations.

  Args:
    patient_agg_obs: An output from `patient_query.find_patient_aggregates()`.
    VL_code: The code for viral load values.
    failure_threshold: VL count threshold of failure.
    end_date: The string representation of the last date as 'YYYY-MM-DD'.
  Returns:
    The aggregated DataFrame.
  """
  end_date = datetime.today()
  if end_date_str:
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
  temp_df = patient_agg_obs[(patient_agg_obs['code'] == VL_code)].copy()
  # Note the above copy is used to avoid setting a new column on a slice next:
  temp_df['sup_VL'] = (temp_df['max_value'] < failure_threshold)
  temp_df['buckets'] = temp_df.apply(
      lambda x: _agg_buckets(x.birthDate, x.gender, end_date), axis=1)
  temp_df_exp = temp_df.explode('buckets')
  temp_df_exp = temp_df_exp.groupby(['sup_VL', 'buckets'], as_index=False)\
      .count()[['sup_VL', 'buckets', 'patientId']]\
      .rename(columns={'patientId': 'count'})
  # calculate ratio
  num_patients = len(temp_df.index)
  temp_df_exp['ratio'] = temp_df_exp['count']/num_patients
  return temp_df_exp
    
calc_TX_PVLS(agg_df, _VL_CODE, 10000, end_date_str='2020-12-30')

,sup_VL,buckets,count,ratio
0,False,15-19_ALL-GENDERS,43,0.016294
1,False,15-19_female,22,0.008336
2,False,15-19_male,21,0.007958
3,False,20-24_ALL-GENDERS,319,0.120879
4,False,20-24_female,145,0.054945
5,False,20-24_male,174,0.065934
6,False,25-49_ALL-GENDERS,1437,0.544524
7,False,25-49_female,748,0.283441
8,False,25-49_male,689,0.261084
9,False,50+_ALL-GENDERS,831,0.314892
